# Animating Open-domain Images with DynamiCrafter and OpenVINO

Animating a still image offers an engaging visual experience. Traditional image animation techniques mainly focus on animating natural scenes with stochastic dynamics (e.g. clouds and fluid) or domain-specific motions (e.g. human hair or body motions), and thus limits their applicability to more general visual content. To overcome this limitation, [DynamiCrafter team](https://doubiiu.github.io/projects/DynamiCrafter/) explores the synthesis of dynamic content for open-domain images, converting them into animated videos. The key idea is to utilize the motion prior of text-to-video diffusion models by incorporating the image into the generative process as guidance. Given an image, DynamiCrafter team first projects it into a text-aligned rich context representation space using a query transformer, which facilitates the video model to digest the image content in a compatible fashion. However, some visual details still struggle to be preserved in the resultant videos. To supplement with more precise image information, DynamiCrafter team further feeds the full image to the diffusion model by concatenating it with the initial noises. Experimental results show that the proposed method can produce visually convincing and more logical & natural motions, as well as higher conformity to the input image.

<table class="center">
  <tr>
    <td colspan="2">"bear playing guitar happily, snowing"</td>
    <td colspan="2">"boy walking on the street"</td>
  </tr>
  <tr>
  <td>
    <img src=https://github.com/Doubiiu/DynamiCrafter/blob/main/assets/showcase/guitar0.jpeg_00.png?raw=True width="170">
  </td>
  <td>
    <img src=https://github.com/Doubiiu/DynamiCrafter/blob/main/assets/showcase/guitar0.gif?raw=True width="170">
  </td>
  <td>
    <img src=https://github.com/Doubiiu/DynamiCrafter/blob/main/assets/showcase/walk0.png_00.png?raw=True width="170">
  </td>
  <td>
    <img src=https://github.com/Doubiiu/DynamiCrafter/blob/main/assets/showcase/walk0.gif?raw=True width="170">
  </td>
  </tr>
</table >


#### Table of contents:
- [Prerequisites](#Prerequisites)
- [Load the original model](#Load-the-original-model)
- [Convert the model to OpenVINO IR](#Convert-the-model-to-OpenVINO-IR)
- [Compiling models](#Compiling-models)
- [Building the pipeline](#Building-the-pipeline)
- [Interactive inference](#Interactive-inference)

## Prerequisites
[back to top ⬆️](#Table-of-contents:)

In [ ]:
%pip uninstall -q -y openvino-dev openvino openvino-nightly
%pip install -q --upgrade openvino-nightly
%pip install -q gradio omegaconf decord einops pytorch_lightning kornia open_clip_torch transformers av torch --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
import sys
from pathlib import Path

if not Path("DynamiCrafter").exists():
    !git clone https://github.com/Doubiiu/DynamiCrafter.git

sys.path.append("DynamiCrafter")

## Load and run the original pipeline
[back to top ⬆️](#Table-of-contents:)

In [ ]:
from scripts.gradio.i2v_test import Image2Video

image2video = Image2Video("results", resolution="256_256")

In [ ]:
model = image2video.model_list[0]

## Convert the model to OpenVINO IR
[back to top ⬆️](#Table-of-contents:)

Let's define the conversion function for PyTorch modules. We use `ov.convert_model` function to obtain OpenVINO Intermediate Representation object and `ov.save_model` function to save it as XML file.

In [ ]:
import torch

import openvino as ov


def convert(model: torch.nn.Module, xml_path: str, example_input, input_shape=None):
    xml_path = Path(xml_path)
    if not xml_path.exists():
        xml_path.parent.mkdir(parents=True, exist_ok=True)
        with torch.no_grad():
            if not input_shape:
                converted_model = ov.convert_model(model, example_input=example_input)
            else:
                converted_model = ov.convert_model(model, example_input=example_input, input=input_shape)
        ov.save_model(converted_model, xml_path, compress_to_fp16=False)

        # cleanup memory
        torch._C._jit_clear_class_registry()
        torch.jit._recursive.concrete_type_store = torch.jit._recursive.ConcreteTypeStore()
        torch.jit._state._clear_class_state()

In [ ]:
from lvdm.modules.encoders.condition import FrozenOpenCLIPEmbedder


COND_STAGE_MODEL_OV_PATH = Path("models/cond_stage_model.xml")


class FrozenOpenCLIPEmbedderWrapper(FrozenOpenCLIPEmbedder):

    def forward(self, tokens):
        z = self.encode_with_transformer(tokens.to(self.device))
        return z


cond_stage_model = FrozenOpenCLIPEmbedderWrapper(device="cpu")


convert(
    cond_stage_model,
    COND_STAGE_MODEL_OV_PATH,
    example_input=torch.ones([1, 77], dtype=torch.long),
)

In [ ]:
ENCODER_FIRST_STAGE_OV_PATH = Path("models/encode_first_stage_ir.xml")


dummy_input = torch.rand([1, 3, 256, 256], dtype=torch.float32)

convert(
    model.first_stage_model.encoder,
    ENCODER_FIRST_STAGE_OV_PATH,
    example_input=dummy_input,
)

In [ ]:
EMBEDDER_OV_PATH = Path("models/embedder_ir.xml")


dummy_input = torch.rand([1, 3, 767, 767], dtype=torch.float32)

convert(model.embedder, EMBEDDER_OV_PATH, example_input=dummy_input, input_shape=[1, 3, -1, -1])

In [ ]:
MODEL_OV_PATH = Path("models/model_ir.xml")


class ModelWrapper(torch.nn.Module):
    def __init__(self, diffusion_model):
        super().__init__()
        self.diffusion_model = diffusion_model

    def forward(self, xc, t, context=None, fs=None, temporal_length=None):
        outputs = self.diffusion_model(xc, t, context=context, fs=fs, temporal_length=temporal_length)
        return outputs


convert(
    ModelWrapper(model.model.diffusion_model),
    MODEL_OV_PATH,
    example_input={
        "xc": torch.rand([1, 8, 16, 32, 32], dtype=torch.float32),
        "t": torch.tensor([1]),
        "context": torch.rand([1, 333, 1024], dtype=torch.float32),
        # 'x_noisy': torch.rand([1, 4, 16, 32, 32], dtype=torch.float32),
        # 'c_crossattn': torch.rand([1, 333, 1024], dtype=torch.float32),
        # 'c_concat': torch.rand([1, 4, 16, 32, 32], dtype=torch.float32),
        "fs": torch.tensor([3]),
        "temporal_length": torch.tensor([16]),
    },
)

`Decoder` receives a `bfloat16` tensor. numpy doesn't support this type. To avoid problems with the conversion lets replace `decode` method to convert bfloat16 to float32.

In [ ]:
import types


def decode(self, z, **kwargs):
    z = self.post_quant_conv(z)
    z = z.float()
    dec = self.decoder(z)
    return dec


model.first_stage_model.decode = types.MethodType(decode, model.first_stage_model)

In [ ]:
DECODER_FIRST_STAGE_OV_PATH = Path("models/decoder_first_stage_ir.xml")


dummy_input = torch.rand([16, 4, 32, 32], dtype=torch.float32)

convert(
    model.first_stage_model.decoder,
    DECODER_FIRST_STAGE_OV_PATH,
    example_input=dummy_input,
)

## Compiling models
[back to top ⬆️](#Table-of-contents:)

Select device from dropdown list for running inference using OpenVINO.

In [ ]:
import ipywidgets as widgets

core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value="AUTO",
    description="Device:",
    disabled=False,
)

device

In [ ]:
compiled_cond_stage_model = core.compile_model(COND_STAGE_MODEL_OV_PATH, device.value)
compiled_encode_first_stage = core.compile_model(ENCODER_FIRST_STAGE_OV_PATH, device.value)
compiled_embedder = core.compile_model(EMBEDDER_OV_PATH, device.value)
compiled_model = core.compile_model(MODEL_OV_PATH, device.value)
compiled_decoder_first_stage = core.compile_model(DECODER_FIRST_STAGE_OV_PATH, device.value)

## Building the pipeline
[back to top ⬆️](#Table-of-contents:)

Let's create callable wrapper classes for compiled models to allow interaction with original pipelines. Note that all of wrapper classes return `torch.Tensor`s instead of `np.array`s.

In [ ]:
import open_clip


class CondStageModelWrapper(torch.nn.Module):
    def __init__(self, cond_stage_model):
        super().__init__()
        self.cond_stage_model = cond_stage_model

    def encode(self, tokens):
        if isinstance(tokens, list):
            tokens = open_clip.tokenize(tokens[0])
        outs = self.cond_stage_model(tokens)[0]

        return torch.from_numpy(outs)


class EncoderFirstStageModelWrapper(torch.nn.Module):
    def __init__(self, encode_first_stage):
        super().__init__()
        self.encode_first_stage = encode_first_stage

    def forward(self, x):
        outs = self.encode_first_stage(x)[0]

        return torch.from_numpy(outs)


class EmbedderWrapper(torch.nn.Module):
    def __init__(self, embedder):
        super().__init__()
        self.embedder = embedder

    def forward(self, x):
        outs = self.embedder(x)[0]

        return torch.from_numpy(outs)


class CModelWrapper(torch.nn.Module):
    def __init__(self, diffusion_model, out_channels):
        super().__init__()
        self.diffusion_model = diffusion_model
        self.out_channels = out_channels

    def forward(self, xc, t, context, fs, temporal_length):
        inputs = {
            "xc": xc,
            "t": t,
            "context": context,
            "fs": fs,
        }
        outs = self.diffusion_model(inputs)[0]

        return torch.from_numpy(outs)


class DecoderFirstStageModelWrapper(torch.nn.Module):
    def __init__(self, decoder_first_stage):
        super().__init__()
        self.decoder_first_stage = decoder_first_stage

    def forward(self, x):
        x.float()
        outs = self.decoder_first_stage(x)[0]

        return torch.from_numpy(outs)

And insert wrappers instances in the pipeline:

In [ ]:
model.cond_stage_model = CondStageModelWrapper(compiled_cond_stage_model)
model.first_stage_model.encoder = EncoderFirstStageModelWrapper(compiled_encode_first_stage)
model.embedder = EmbedderWrapper(compiled_embedder)
model.model.diffusion_model = CModelWrapper(compiled_model, model.model.diffusion_model.out_channels)
model.first_stage_model.decoder = DecoderFirstStageModelWrapper(compiled_decoder_first_stage)

In [ ]:
import time

from einops import repeat
from pytorch_lightning import seed_everything
import torchvision.transforms as transforms

from scripts.evaluation.funcs import save_videos, batch_ddim_sampling, get_latent_z


def get_image(image, prompt, model, steps=5, cfg_scale=7.5, eta=1.0, fs=3, seed=123):
    result_dir = "results"
    seed_everything(seed)
    transform = transforms.Compose(
        [
            transforms.Resize(min((256, 256))),
            transforms.CenterCrop((256, 256)),
        ]
    )
    # torch.cuda.empty_cache()
    print("start:", prompt, time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time())))
    start = time.time()
    if steps > 60:
        steps = 60
    model = model.cpu()
    batch_size = 1
    channels = model.model.diffusion_model.out_channels
    frames = model.temporal_length
    h, w = 256 // 8, 256 // 8
    noise_shape = [batch_size, channels, frames, h, w]

    # text cond
    with torch.no_grad(), torch.cpu.amp.autocast():
        text_emb = model.get_learned_conditioning([prompt])

        # img cond
        img_tensor = torch.from_numpy(image).permute(2, 0, 1).float().to(model.device)
        img_tensor = (img_tensor / 255.0 - 0.5) * 2

        image_tensor_resized = transform(img_tensor)  # 3,h,w
        videos = image_tensor_resized.unsqueeze(0)  # bchw

        z = get_latent_z(model, videos.unsqueeze(2))  # bc,1,hw

        img_tensor_repeat = repeat(z, "b c t h w -> b c (repeat t) h w", repeat=frames)

        cond_images = model.embedder(img_tensor.unsqueeze(0))  # blc

        img_emb = model.image_proj_model(cond_images)

        imtext_cond = torch.cat([text_emb, img_emb], dim=1)

        fs = torch.tensor([fs], dtype=torch.long, device=model.device)
        cond = {"c_crossattn": [imtext_cond], "fs": fs, "c_concat": [img_tensor_repeat]}

        ## inference
        batch_samples = batch_ddim_sampling(model, cond, noise_shape, n_samples=1, ddim_steps=steps, ddim_eta=eta, cfg_scale=cfg_scale)
        ## b,samples,c,t,h,w
        prompt_str = prompt.replace("/", "_slash_") if "/" in prompt else prompt
        prompt_str = prompt_str.replace(" ", "_") if " " in prompt else prompt_str
        prompt_str = prompt_str[:40]
        if len(prompt_str) == 0:
            prompt_str = "empty_prompt"

    save_videos(batch_samples, result_dir, filenames=[prompt_str], fps=8)
    print(f"Saved in {prompt_str}. Time used: {(time.time() - start):.2f} seconds")

    return os.path.join(result_dir, f"{prompt_str}.mp4")

## Interactive inference
[back to top ⬆️](#Table-of-contents:)

In [ ]:
import os

import gradio as gr
from scripts.gradio.i2v_test import Image2Video


i2v_examples_256 = [
    ["DynamiCrafter/prompts/256/art.png", "man fishing in a boat at sunset", 50, 7.5, 1.0, 3, 234],
    ["DynamiCrafter/prompts/256/boy.png", "boy walking on the street", 50, 7.5, 1.0, 3, 125],
    ["DynamiCrafter/prompts/256/dance1.jpeg", "two people dancing", 50, 7.5, 1.0, 3, 116],
    ["DynamiCrafter/prompts/256/fire_and_beach.jpg", "a campfire on the beach and the ocean waves in the background", 50, 7.5, 1.0, 3, 111],
    ["DynamiCrafter/prompts/256/guitar0.jpeg", "bear playing guitar happily, snowing", 50, 7.5, 1.0, 3, 122],
]


def dynamicrafter_demo(result_dir="./tmp/"):
    resolution = "256_256"
    css = """#input_img {max-width: 256px !important} #output_vid {max-width: 256px; max-height: 256px}"""

    image2video = Image2Video(result_dir, resolution=resolution)
    image2video.model_list = [model]
    with gr.Blocks(analytics_enabled=False, css=css) as dynamicrafter_iface:
        with gr.Tab(label="Image2Video_256x256"):
            with gr.Column():
                with gr.Row():
                    with gr.Column():
                        with gr.Row():
                            i2v_input_image = gr.Image(label="Input Image", elem_id="input_img")
                        with gr.Row():
                            i2v_input_text = gr.Text(label="Prompts")
                        with gr.Row():
                            i2v_seed = gr.Slider(label="Random Seed", minimum=0, maximum=10000, step=1, value=123)
                            i2v_eta = gr.Slider(minimum=0.0, maximum=1.0, step=0.1, label="ETA", value=1.0, elem_id="i2v_eta")
                            i2v_cfg_scale = gr.Slider(minimum=1.0, maximum=15.0, step=0.5, label="CFG Scale", value=7.5, elem_id="i2v_cfg_scale")
                        with gr.Row():
                            i2v_steps = gr.Slider(minimum=1, maximum=60, step=1, elem_id="i2v_steps", label="Sampling steps", value=50)
                            i2v_motion = gr.Slider(minimum=1, maximum=4, step=1, elem_id="i2v_motion", label="Motion magnitude", value=3)
                        i2v_end_btn = gr.Button("Generate")
                    with gr.Row():
                        i2v_output_video = gr.Video(label="Generated Video", elem_id="output_vid", autoplay=True, show_share_button=True)

                gr.Examples(
                    examples=i2v_examples_256,
                    inputs=[i2v_input_image, i2v_input_text, model, i2v_steps, i2v_cfg_scale, i2v_eta, i2v_motion, i2v_seed],
                    outputs=[i2v_output_video],
                    fn=get_image,
                    cache_examples=False,
                )
            i2v_end_btn.click(
                inputs=[i2v_input_image, i2v_input_text, model, i2v_steps, i2v_cfg_scale, i2v_eta, i2v_motion, i2v_seed],
                outputs=[i2v_output_video],
                fn=get_image,
            )

    return dynamicrafter_iface


demo = dynamicrafter_demo("results")


try:
    demo.queue().launch(debug=True)
except Exception:
    demo.queue().launch(debug=True, share=True)
# if you are launching remotely, specify server_name and server_port
# demo.launch(server_name='your server name', server_port='server port in int')
# Read more in the docs: https://gradio.app/docs/